In [48]:
from scipy.stats import chi2_contingency
import pandas as pd
import numpy as np

신규고객과 기존고객의 동질성 비교
1. 개인정보 : 성별(가입, 구입), 연령(가입, 구입), 국가(가입, 구입), 유입경로(가입, 구입)
2. 구매정보 : 상위카테고리, 상위 브랜드
3. 구매경험 : 가입후 첫 결제 기간

In [49]:
look = pd.read_csv('C:/Users/LG/Desktop/데이터분석/프로젝트/최종프로젝트/3차/looker.csv')

C:\Users\LG\AppData\Local\Temp\ipykernel_17532\851929299.py:1: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  look = pd.read_csv('C:/Users/LG/Desktop/데이터분석/프로젝트/최종프로젝트/3차/looker.csv')


In [50]:
# 형변환
def convert_type(df):
    for col in df.columns:
        if '_at' in col:    # 시간 데이터
            df = df.assign(**{col:pd.to_datetime(df[col])})
            print('[+] {0:<30} >>> datetime'.format(col))
        elif '_id' in col:  # id 컬럼 -> obj형으로
            df = df.assign(**{col:df[col].astype('object')})
            print('[+] {0:<30} >>> object'.format(col))
    return df

In [51]:
look = convert_type(look)

[+] user_id                        >>> object
[+] account_created_at             >>> datetime
[+] order_id                       >>> object
[+] order_items_id                 >>> object
[+] order_created_at               >>> datetime
[+] session_id                     >>> object
[+] login_user_id                  >>> object
[+] sess_created_at                >>> datetime
[+] inventory_item_id              >>> object
[+] product_id                     >>> object
[+] inventory_created_at           >>> datetime
[+] inventory_sold_at              >>> datetime


In [52]:
pd.set_option('display.max_columns', None)

In [53]:
df = look.copy()

In [54]:
df.describe(include='all')

,user_id,age,gender,account_state,account_city,account_country,latitude,longitude,account_traffic_source,account_created_at,order_id,order_items_id,status,sale_price,order_created_at,session_id,sequence_number,login_user_id,sess_created_at,sess_traffic_source,uri,event_type,ip_address,sess_state,sess_city,inventory_item_id,product_id,category,brand,product_name,inventory_created_at,inventory_sold_at,cost
count,102365.0,102365.000000,102365,102365,101376,102365,102365.000000,102365.000000,102365,102365,82409.0,82409.0,82409,82409.000000,82409,2363176,2.363176e+06,1237505.0,2363176,2363176,2363176,2363176,2363176,2363176,2340709,490705.0,490705.0,490705,490304,490676,490705,181759,490705.000000
unique,75608.0,NaN,2,227,7483,16,NaN,NaN,5,NaN,73703.0,82409.0,3,NaN,NaN,681759,NaN,80044.0,NaN,5,35530,6,681683,231,8775,490705.0,29046.0,26,2752,27236,NaN,NaN,NaN
top,51531.0,NaN,F,Guangdong,Shanghai,China,NaN,NaN,Search,NaN,86789.0,134592.0,Shipped,NaN,NaN,f31ff7c4-1e26-4a60-a32d-3935a82456d1,NaN,32996.0,NaN,Email,/cart,product,144.18.158.227,Guangdong,Shanghai,363366.0,18795.0,Intimates,Allegra K,Wrangler Men's Premium Performance Cowboy Cut ...,NaN,NaN,NaN
freq,10.0,NaN,51265,5426,2588,35066,NaN,NaN,71750,NaN,4.0,1.0,38042,NaN,NaN,13,NaN,155.0,NaN,1061140,595994,845607,16,126080,58830,1.0,58.0,36353,17019,161,NaN,NaN,NaN
mean,NaN,41.074762,NaN,NaN,NaN,NaN,28.360225,25.074985,NaN,2021-07-31 07:15:22.991257856,NaN,NaN,NaN,59.796109,2022-10-23 21:15:30.087734016,NaN,3.144442e+00,NaN,2022-03-13 08:11:34.828995072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-14 10:35:30.236292608,2022-10-24 07:44:52.389372416,28.658986
min,NaN,12.000000,NaN,NaN,NaN,NaN,-43.007536,-158.164931,NaN,2019-01-02 00:00:00,NaN,NaN,NaN,0.020000,2019-01-06 00:00:00,NaN,1.000000e+00,NaN,2019-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-08 00:00:00,2019-01-06 00:00:00,0.008300
25%,NaN,26.000000,NaN,NaN,NaN,NaN,26.222996,-50.955736,NaN,2020-04-15 00:00:00,NaN,NaN,NaN,24.500000,2022-02-26 00:00:00,NaN,1.000000e+00,NaN,2021-02-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-16 00:00:00,2022-02-24 00:00:00,11.368000
50%,NaN,41.000000,NaN,NaN,NaN,NaN,35.211528,4.850559,NaN,2021-08-02 00:00:00,NaN,NaN,NaN,39.990002,2023-02-19 00:00:00,NaN,2.000000e+00,NaN,2022-07-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-08 00:00:00,2023-02-21 00:00:00,19.832000
75%,NaN,56.000000,NaN,NaN,NaN,NaN,40.715959,116.376103,NaN,2022-11-13 00:00:00,NaN,NaN,NaN,69.949997,2023-09-27 00:00:00,NaN,4.000000e+00,NaN,2023-06-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-13 00:00:00,2023-09-27 00:00:00,34.550301
max,NaN,70.000000,NaN,NaN,NaN,NaN,64.865194,153.560238,NaN,2024-01-16 00:00:00,NaN,NaN,NaN,999.000000,2024-01-20 00:00:00,NaN,1.300000e+01,NaN,2024-01-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-21 00:00:00,2024-01-21 00:00:00,557.151002


- cut 함수 기본 사용법
- 기능: 연속적인 데이터 값을 지정된 구간으로 나누어 범주화합니다.
- 주요 파라미터:
- x: 나누고자 하는 데이터 배열.
- bins: 구간의 경계값이나 구간의 수를 지정.
- labels: 구간별로 지정할 라벨. 지정하지 않으면 구간의 범위가 라벨로 사용됩니다.
- right: 구간의 오른쪽을 포함할지 여부. 기본값은 True.


In [55]:
bins = [10,19,29,39,49,59,69,79]
bins_names = [10,20,30,40,50,60,70]

In [56]:
df['age2'] = pd.cut(df['age'], bins, labels = bins_names)

In [57]:
df['age2'].value_counts()

age2
60    17591
50    17450
20    17422
40    17251
30    17074
10    13846
70     1731
Name: count, dtype: int64

In [58]:
df[df['age']==70][['age','age2']]

,age,age2
740,70.0,70
1441,70.0,70
1963,70.0,70
2215,70.0,70
2446,70.0,70
...,...,...
2835053,70.0,70
2835064,70.0,70
2837743,70.0,70
2839313,70.0,70


In [59]:
# 실험군: 2024 신규 유입 유저
treatment = df[df.account_created_at >= '2024-01-11'].copy()

In [60]:
# 대조군: 기존 유저
control = df[df.account_created_at < '2024-01-11'].copy()

In [61]:
print(
    f'전체 유저 수 : {df.user_id.nunique(dropna=True)}명\n\
실험군 (treatment) : {treatment.user_id.nunique()}명\n\
대조군 (control) : {control.user_id.nunique()}명'
    )

전체 유저 수 : 75608명
실험군 (treatment) : 2108명
대조군 (control) : 73500명


1. 귀무가설 : 2024-1-11전후로 가입한 고객의 연령에는 차이가 없다

In [62]:
# 실험군의 연령별 가입자수
test_10 = treatment[treatment['age2']==10]['user_id'].nunique()
test_20 = treatment[treatment['age2']==20]['user_id'].nunique()
test_30 = treatment[treatment['age2']==30]['user_id'].nunique()
test_40 = treatment[treatment['age2']==40]['user_id'].nunique()
test_50 = treatment[treatment['age2']==50]['user_id'].nunique()
test_60 = treatment[treatment['age2']==60]['user_id'].nunique()
test_70 = treatment[treatment['age2']==70]['user_id'].nunique()

# 대조군의 연령별 가입자수
cont_10 = control[control['age2']==10]['user_id'].nunique()
cont_20 = control[control['age2']==20]['user_id'].nunique()
cont_30 = control[control['age2']==30]['user_id'].nunique()
cont_40 = control[control['age2']==40]['user_id'].nunique()
cont_50 = control[control['age2']==50]['user_id'].nunique()
cont_60 = control[control['age2']==60]['user_id'].nunique()
cont_70 = control[control['age2']==70]['user_id'].nunique()

## Part1. Chi-square Test Report
age10 = [test_10, cont_10] # A와 B의 10대 유저 수
age20 = [test_20, cont_20] # A와 B의 20대 유저 수
age30 = [test_30, cont_30] # A와 B의 30대 유저 수
age40 = [test_40, cont_40] # A와 B의 40대 유저 수
age50 = [test_50, cont_50] # A와 B의 50대 유저 수
age60 = [test_60, cont_60] # A와 B의 60대 유저 수
age70 = [test_70, cont_70] # A와 B의 70대 유저 수

In [63]:
## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([age10,age20,age30,age40,age50,age60,age70], columns=['treatment', 'control'], index=["10대",'20대',"30대",'40대',"50대",'60대','70대'])
cont_table

,treatment,control
10대,284,9898
20대,361,12492
30대,306,12281
40대,337,12417
50대,409,12581
60대,372,12561
70대,39,1270


In [64]:
## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([age10,age20,age30,age40,age50,age60,age70])

In [65]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ["10대",'20대',"30대",'40대',"50대",'60대','70대'])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 13.72
P-value: 0.03
--------------------------
Expected Values
      treatment       control
10대  283.880753   9898.119247
20대  358.349963  12494.650037
30대  350.933711  12236.066289
40대  355.589779  12398.410221
50대  362.169612  12627.830388
60대  360.580415  12572.419585
70대   36.495768   1272.504232
--------------------------
Observed Values
     treatment  control
10대        284     9898
20대        361    12492
30대        306    12281
40대        337    12417
50대        409    12581
60대        372    12561
70대         39     1270
 


1-1. 2024-1-11전후로 가입한 고객중 가입자의 연령 결과(차이가 있다)
- 유의성검정 : P-value가 0.03로 0.05보다 작으므로 귀무가설을 기각한다

1-2. 귀무가설 : 2024-1-11전후로 가입한 고객중 구매자의 연령은 차이가 없을 것이다. 

In [66]:
# 실험군: 2024 신규 유입 유저 + 구매자
treatment2 = df[(df['account_created_at'] >= '2024-01-11')&(df['event_type'] =='purchase')]
control2 = df[(df['account_created_at'] < '2024-01-11')&(df['event_type'] =='purchase')]

In [67]:
# 실험군의 연령별 가입자수
test_10 = treatment2[treatment2['age2']==10]['user_id'].nunique()
test_20 = treatment2[treatment2['age2']==20]['user_id'].nunique()
test_30 = treatment2[treatment2['age2']==30]['user_id'].nunique()
test_40 = treatment2[treatment2['age2']==40]['user_id'].nunique()
test_50 = treatment2[treatment2['age2']==50]['user_id'].nunique()
test_60 = treatment2[treatment2['age2']==60]['user_id'].nunique()
test_70 = treatment2[treatment2['age2']==70]['user_id'].nunique()

# 대조군의 연령별 가입자수
cont_10 = control2[control2['age2']==10]['user_id'].nunique()
cont_20 = control2[control2['age2']==20]['user_id'].nunique()
cont_30 = control2[control2['age2']==30]['user_id'].nunique()
cont_40 = control2[control2['age2']==40]['user_id'].nunique()
cont_50 = control2[control2['age2']==50]['user_id'].nunique()
cont_60 = control2[control2['age2']==60]['user_id'].nunique()
cont_70 = control2[control2['age2']==70]['user_id'].nunique()

## Part1. Chi-square Test Report
age10 = [test_10, cont_10] # A와 B의 10대 유저 수
age20 = [test_20, cont_20] # A와 B의 20대 유저 수
age30 = [test_30, cont_30] # A와 B의 30대 유저 수
age40 = [test_40, cont_40] # A와 B의 40대 유저 수
age50 = [test_50, cont_50] # A와 B의 50대 유저 수
age60 = [test_60, cont_60] # A와 B의 60대 유저 수
age70 = [test_70, cont_70] # A와 B의 70대 유저 수

In [68]:
## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([age10,age20,age30,age40,age50,age60,age70], columns=['treatment', 'control'], index=["10대",'20대',"30대",'40대',"50대",'60대','70대'])

In [69]:
cont_table

,treatment,control
10대,57,2077
20대,83,2573
30대,68,2645
40대,70,2625
50대,75,2605
60대,71,2685
70대,11,261


In [70]:
## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([age10,age20,age30,age40,age50,age60,age70])

In [71]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ["10대",'20대',"30대",'40대',"50대",'60대','70대'])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 4.33
P-value: 0.63
--------------------------
Expected Values
     treatment      control
10대  58.360996  2075.639004
20대  72.636741  2583.363259
30대  74.195587  2638.804413
40대  73.703320  2621.296680
50대  73.293097  2606.706903
60대  75.371558  2680.628442
70대   7.438702   264.561298
--------------------------
Observed Values
     treatment  control
10대         57     2077
20대         83     2573
30대         68     2645
40대         70     2625
50대         75     2605
60대         71     2685
70대         11      261
 


1-2. 2024-1-11전후로 가입한 고객중 구매자의 연령 결과(차이가 없다)
- 유의성검정 : P-value가 0.63로 0.05보다 크므로 귀무가설을 채택한다

2-1.성별에 따른 가입자수 상관관계 구하기

In [72]:
treatment.head(1)

,user_id,age,gender,account_state,account_city,account_country,latitude,longitude,account_traffic_source,account_created_at,order_id,order_items_id,status,sale_price,order_created_at,session_id,sequence_number,login_user_id,sess_created_at,sess_traffic_source,uri,event_type,ip_address,sess_state,sess_city,inventory_item_id,product_id,category,brand,product_name,inventory_created_at,inventory_sold_at,cost,age2
1711,65932.0,35.0,F,Ningxia Hui Autonomous Region,Rizhao,China,37.990152,106.165456,Organic,2024-01-15,82634.0,119972.0,Complete,49.990002,2024-01-16,43f18013-17af-4eeb-934a-7dce5eef8398,10.0,65932.0,2024-01-16,Adwords,/purchase,purchase,79.86.248.35,Ningxia Hui Autonomous Region,Rizhao,323796.0,1637.0,Fashion Hoodies & Sweatshirts,Ed Hardy,Ed Hardy Womens Rope & Tiger Hoodie - Mustard,2021-11-04,NaT,25.044991,30


In [73]:
# 실험군의 성별 가입자수
test_F = treatment[treatment['gender']=='M']['user_id'].nunique()
test_M = treatment[treatment['gender']=='F']['user_id'].nunique()
# 대조군의 성별 가입자수
cont_F = control[control['gender']=='M']['user_id'].nunique()
cont_M = control[control['gender']=='F']['user_id'].nunique()

## Part1. Chi-square Test Report
genderF = [test_F, cont_F] # A와 B의 여성 유저 수
genderM = [test_M, cont_M] # A와 B의 남성 유저 수

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([genderF,genderM], columns=['treatment', 'control'], index=['F','M'])

In [74]:
cont_table

,treatment,control
F,1057,36640
M,1051,36860


In [75]:
## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([genderF,genderM])

In [76]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ['F','M'])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 0.06
P-value: 0.81
--------------------------
Expected Values
     treatment       control
F  1051.016771  36645.983229
M  1056.983229  36854.016771
--------------------------
Observed Values
   treatment  control
F       1057    36640
M       1051    36860
 


2-1. 성별에 따른 가입자수 상관관계 결과(차이가 없다)
- 유의성검정 : P-value가 0.81로 0.05보다 크므로 귀무가설을 채택한다(차이가 없다)
- 기각역 : 검종통계량 0.06은 기각역(3.838)보다 작으므로 귀무가설 채택역에 포함된다

2-2. 2024-1-11전후로 가입한 고객중 구매자의 성별의 차이

In [77]:
# 실험군의 성별 가입자수
test_F = treatment2[treatment2['gender']=='M']['user_id'].nunique()
test_M = treatment2[treatment2['gender']=='F']['user_id'].nunique()
# 대조군의 성별 가입자수
cont_F = control2[control2['gender']=='M']['user_id'].nunique()
cont_M = control2[control2['gender']=='F']['user_id'].nunique()

## Part1. Chi-square Test Report
genderF = [test_F, cont_F] # A와 B의 여성 유저 수
genderM = [test_M, cont_M] # A와 B의 남성 유저 수

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([genderF,genderM], columns=['treatment', 'control'], index=['F','M'])
cont_table

,treatment,control
F,222,7679
M,213,7792


In [78]:
## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([genderF,genderM])

## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ['F','M'])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 0.28
P-value: 0.6
--------------------------
Expected Values
    treatment      control
F  216.077895  7684.922105
M  218.922105  7786.077895
--------------------------
Observed Values
   treatment  control
F        222     7679
M        213     7792
 


2-2. 2024-1-11전후로 가입한 고객중 구매자의 성별 결과(차이가 없다)
-  유의성검정 : P-value가 0.28로 0.05보다 작으므로 귀무가설을 채택한다

3-1. 국가에 따른 가입자수 상관관계

In [79]:
treatment['account_country'].value_counts()

account_country
China             1001
United States      652
Brasil             463
South Korea        139
France             123
United Kingdom     118
Germany            117
Spain              108
Japan               81
Australia           60
Belgium             40
Poland               5
Colombia             1
Name: count, dtype: int64

In [80]:
# 실험군의 국가별 가입자수
test_China = treatment[treatment['account_country']=='China']['user_id'].nunique()
test_United_States = treatment[treatment['account_country']=='United States']['user_id'].nunique()
test_Brasil =treatment[treatment['account_country']=='Brasil']['user_id'].nunique()
test_South_Korea =treatment[treatment['account_country']=='South Korea']['user_id'].nunique()
test_United_Kingdom = treatment[treatment['account_country']=='United Kingdom']['user_id'].nunique()
test_Germany = treatment[treatment['account_country']=='Germany']['user_id'].nunique()
test_France =treatment[treatment['account_country']=='France']['user_id'].nunique()
test_Spain = treatment[treatment['account_country']=='Spain']['user_id'].nunique()
test_Japan = treatment[treatment['account_country']=='Japan']['user_id'].nunique()
test_Australia = treatment[treatment['account_country']=='Australia']['user_id'].nunique()
test_Belgium = treatment[treatment['account_country']=='Poland']['user_id'].nunique()
test_Poland  = treatment[treatment['account_country']=='Spain']['user_id'].nunique()
test_Colombia  = treatment[treatment['account_country']=='Colombia']['user_id'].nunique()
# 대조군의 국가별 가입자수
cont_China = control[control['account_country']=='China']['user_id'].nunique()
cont_United_States = control[control['account_country']=='United States']['user_id'].nunique()
cont_Brasil = control[control['account_country']=='Brasil']['user_id'].nunique()
cont_South_Korea = control[control['account_country']=='South Korea']['user_id'].nunique()
cont_United_Kingdom = control[control['account_country']=='United Kingdom']['user_id'].nunique()
cont_Germany = control[control['account_country']=='Germany']['user_id'].nunique()
cont_France = control[control['account_country']=='France']['user_id'].nunique()
cont_Spain = control[control['account_country']=='Spain']['user_id'].nunique()
cont_Japan = control[control['account_country']=='Japan']['user_id'].nunique()
cont_Australia =control[control['account_country']=='Australia']['user_id'].nunique()
cont_Belgium =control[control['account_country']=='Poland']['user_id'].nunique()
cont_Poland  = control[control['account_country']=='Spain']['user_id'].nunique()
cont_Colombia  = control[control['account_country']=='Colombia']['user_id'].nunique()

## Part1. Chi-square Test Report
China  = [test_China,cont_China]  
United_States  = [test_United_States ,cont_United_States ]  
Brasil  = [test_Brasil ,cont_Brasil ]  
South_Korea = [test_South_Korea,cont_South_Korea]   
United_Kingdom = [test_United_Kingdom,cont_United_Kingdom]   
Germany  = [test_Germany,cont_Germany]  
France  = [test_France ,cont_France ]  
Spain  = [test_Spain,cont_Spain]  
Japan    = [test_Japan ,cont_Japan ]      
Australia   = [test_Australia,cont_Australia]  
Belgium  = [test_Belgium,cont_Belgium]  
Poland  = [test_Poland,cont_Poland]  
Colombia  = [test_Colombia,cont_Colombia]  

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([China,United_States,Brasil ,South_Korea    ,United_Kingdom , Germany   ,
France  ,Spain,Japan  ,Australia  ,Belgium,Poland   , Colombia  ], columns=['treatment', 'control'], index=['China', 'United States', 'Brasil', 'South Korea', 'United Kingdom',
       'Germany', 'France', 'Spain', 'Japan', 'Australia', 'Belgium', 'Poland',
       'Colombia'])

In [81]:
## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([China,United_States,Brasil ,South_Korea    ,United_Kingdom , Germany   ,
France  ,Spain,Japan  ,Australia  ,Belgium,Poland   , Colombia  ])

In [82]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ['China', 'United States', 'Brasil', 'South Korea', 'United Kingdom',
       'Germany', 'France', 'Spain', 'Japan', 'Australia', 'Belgium', 'Poland',
       'Colombia'])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 11.48
P-value: 0.49
--------------------------
Expected Values
                 treatment       control
China           717.858343  25096.141657
United States   474.224774  16578.775226
Brasil          304.284729  10637.715271
South Korea     111.986346   3915.013654
United Kingdom   96.691464   3380.308536
Germany          86.902740   3038.097260
France           98.109717   3429.890283
Spain            84.705838   2961.294162
Japan            51.752320   1809.247680
Australia        44.605438   1559.394562
Belgium           4.838745    169.161255
Poland           84.705838   2961.294162
Colombia          0.333707     11.666293
--------------------------
Observed Values
                treatment  control
China                 723    25091
United States         461    16592
Brasil                335    10607
South Korea           103     3924
United Kingdom         80     3397
Germany                84     3041
France                 9

3-1. 나라에 따른 가입자수 상관관계 결과 (차이가 없다)
- 유의성검정 : P-value가 0.49로 0.05보다 크므로 귀무가설 채택
- 가설검정 : 카이제곱의 값11.4 으로 기각역(자유도12)21.03보다 작으므로 채택한다


3-2. 2024-1-11전후로 가입한 고객중 구매자의 국가에 대한 차이

In [83]:
# 실험군의 국가별 가입자수
test_China = treatment2[treatment2['account_country']=='China']['user_id'].nunique()
test_United_States = treatment2[treatment2['account_country']=='United States']['user_id'].nunique()
test_Brasil =treatment2[treatment2['account_country']=='Brasil']['user_id'].nunique()
test_South_Korea =treatment2[treatment2['account_country']=='South Korea']['user_id'].nunique()
test_United_Kingdom = treatment2[treatment2['account_country']=='United Kingdom']['user_id'].nunique()
test_Germany = treatment2[treatment2['account_country']=='Germany']['user_id'].nunique()
test_France =treatment2[treatment2['account_country']=='France']['user_id'].nunique()
test_Spain = treatment2[treatment2['account_country']=='Spain']['user_id'].nunique()
test_Japan = treatment2[treatment2['account_country']=='Japan']['user_id'].nunique()
test_Australia = treatment2[treatment2['account_country']=='Australia']['user_id'].nunique()
test_Belgium = treatment2[treatment2['account_country']=='Poland']['user_id'].nunique()
test_Poland  = treatment2[treatment2['account_country']=='Spain']['user_id'].nunique()
test_Colombia  = treatment2[treatment2['account_country']=='Colombia']['user_id'].nunique()
# 대조군의 국가별 가입자수
cont_China = control2[control2['account_country']=='China']['user_id'].nunique()
cont_United_States = control2[control2['account_country']=='United States']['user_id'].nunique()
cont_Brasil = control2[control2['account_country']=='Brasil']['user_id'].nunique()
cont_South_Korea = control2[control2['account_country']=='South Korea']['user_id'].nunique()
cont_United_Kingdom = control2[control2['account_country']=='United Kingdom']['user_id'].nunique()
cont_Germany = control2[control2['account_country']=='Germany']['user_id'].nunique()
cont_France = control2[control2['account_country']=='France']['user_id'].nunique()
cont_Spain = control2[control2['account_country']=='Spain']['user_id'].nunique()
cont_Japan = control2[control2['account_country']=='Japan']['user_id'].nunique()
cont_Australia =control2[control2['account_country']=='Australia']['user_id'].nunique()
cont_Belgium =control2[control2['account_country']=='Poland']['user_id'].nunique()
cont_Poland  = control2[control2['account_country']=='Spain']['user_id'].nunique()
cont_Colombia  = control2[control2['account_country']=='Colombia']['user_id'].nunique()

## Part1. Chi-square Test Report
China  = [test_China,cont_China]  
United_States  = [test_United_States ,cont_United_States ]  
Brasil  = [test_Brasil ,cont_Brasil ]  
South_Korea = [test_South_Korea,cont_South_Korea]   
United_Kingdom = [test_United_Kingdom,cont_United_Kingdom]   
Germany  = [test_Germany,cont_Germany]  
France  = [test_France ,cont_France ]  
Spain  = [test_Spain,cont_Spain]  
Japan    = [test_Japan ,cont_Japan ]      
Australia   = [test_Australia,cont_Australia]  
Belgium  = [test_Belgium,cont_Belgium]  
Poland  = [test_Poland,cont_Poland]  
Colombia  = [test_Colombia,cont_Colombia]  

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([China,United_States,Brasil ,South_Korea    ,United_Kingdom , Germany   ,
France  ,Spain,Japan  ,Australia  ,Belgium,Poland   , Colombia  ], columns=['treatment', 'control'], index=['China', 'United States', 'Brasil', 'South Korea', 'United Kingdom',
       'Germany', 'France', 'Spain', 'Japan', 'Australia', 'Belgium', 'Poland',
       'Colombia'])

In [84]:
## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([China,United_States,Brasil ,South_Korea    ,United_Kingdom , Germany   ,
France  ,Spain,Japan  ,Australia  ,Belgium,Poland   , Colombia  ])

## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ['China', 'United States', 'Brasil', 'South Korea', 'United Kingdom',
       'Germany', 'France', 'Spain', 'Japan', 'Australia', 'Belgium', 'Poland',
       'Colombia'])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 8.36
P-value: 0.76
--------------------------
Expected Values
                 treatment      control
China           150.124272  5342.875728
United States    97.322508  3463.677492
Brasil           63.105215  2245.894785
South Korea      23.722532   844.277468
United Kingdom   19.841657   706.158343
Germany          18.666462   664.333538
France           20.169618   717.830382
Spain            16.398064   583.601936
Japan            10.248790   364.751210
Australia         9.018935   320.981065
Belgium           0.929224    33.070776
Poland           16.398064   583.601936
Colombia          0.054660     1.945340
--------------------------
Observed Values
                treatment  control
China                 143     5350
United States         100     3461
Brasil                 78     2231
South Korea            21      847
United Kingdom         19      707
Germany                19      664
France                 13      725
Spai

3-2. 2024-1-11전후로 가입한 고객중 구매자의 국가에 대한 차이가 없다
- 유의성검정 : P-value가 0.76로 0.05보다 크므로 귀무가설 채택

4-1. 유입경로에 따른 가입자수 상관관계

In [85]:
df['sess_traffic_source'].unique()

array([nan, 'Adwords', 'Email', 'Facebook', 'YouTube', 'Organic'],
      dtype=object)

In [86]:
# 실험군의 유입경로 가입자수
test_Email  = treatment[treatment['sess_traffic_source']=='Email']['user_id'].nunique()
test_Adwords  = treatment[treatment['sess_traffic_source']=='Adwords']['user_id'].nunique()
test_Facebook  =treatment[treatment['sess_traffic_source']=='Facebook']['user_id'].nunique()
test_YouTube  =treatment[treatment['sess_traffic_source']=='YouTube']['user_id'].nunique()
test_Organic  = treatment[treatment['sess_traffic_source']=='Organic']['user_id'].nunique()

# 대조군의 유입경로 가입자수
cont_Email = control[control['sess_traffic_source']=='Email']['user_id'].nunique()
cont_Adwords  = control[control['sess_traffic_source']=='Adwords']['user_id'].nunique()
cont_Facebook = control[control['sess_traffic_source']=='Facebook']['user_id'].nunique()
cont_YouTube = control[control['sess_traffic_source']=='YouTube']['user_id'].nunique()
cont_Organic = control[control['sess_traffic_source']=='Organic']['user_id'].nunique()


## Part1. Chi-square Test Report
Email = [test_Email, cont_Email]
Adwords =[test_Adwords, cont_Adwords]
Facebook =[test_Facebook, cont_Facebook]
YouTube =[test_YouTube, cont_YouTube]
Organic =[test_Organic, cont_Organic]

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([Email ,Adwords ,Facebook,YouTube ,Organic ], columns=['treatment', 'control'],index=["Email" ,"Adwords" ,"Facebook","YouTube" ,"Organic"])

In [87]:
## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([Email ,Adwords ,Facebook,YouTube ,Organic ])

In [88]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ["Email" ,"Adwords" ,"Facebook","YouTube" ,"Organic"])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 4.5
P-value: 0.34
--------------------------
Expected Values
           treatment      control
Email     253.496072  8972.503928
Adwords   183.211772  6484.788228
Facebook   68.058722  2408.941278
YouTube    66.684908  2360.315092
Organic    33.548526  1187.451474
--------------------------
Observed Values
          treatment  control
Email           265     8961
Adwords         185     6483
Facebook         56     2421
YouTube          72     2355
Organic          27     1194
 


4-1. 유입경로에 따른 가입자수 상관관계 결과 -> 차이없다
- 유의성검정 : P-value가 0.34로 0.05보다 크므로 귀무가설을 채택한다(차이가 없다)

4-2. 2024-1-11전후로 가입한 고객중 구매자의 국가에 대한 차이가 없을 것이다

In [89]:
# 실험군의 유입경로 가입자수
test_Email  = treatment2[treatment2['sess_traffic_source']=='Email']['user_id'].nunique()
test_Adwords  = treatment2[treatment2['sess_traffic_source']=='Adwords']['user_id'].nunique()
test_Facebook  =treatment2[treatment2['sess_traffic_source']=='Facebook']['user_id'].nunique()
test_YouTube  =treatment2[treatment2['sess_traffic_source']=='YouTube']['user_id'].nunique()
test_Organic  = treatment2[treatment2['sess_traffic_source']=='Organic']['user_id'].nunique()

# 대조군의 유입경로 가입자수
cont_Email = control2[control2['sess_traffic_source']=='Email']['user_id'].nunique()
cont_Adwords  = control2[control2['sess_traffic_source']=='Adwords']['user_id'].nunique()
cont_Facebook = control2[control2['sess_traffic_source']=='Facebook']['user_id'].nunique()
cont_YouTube = control2[control2['sess_traffic_source']=='YouTube']['user_id'].nunique()
cont_Organic = control2[control2['sess_traffic_source']=='Organic']['user_id'].nunique()


## Part1. Chi-square Test Report
Email = [test_Email, cont_Email]
Adwords =[test_Adwords, cont_Adwords]
Facebook =[test_Facebook, cont_Facebook]
YouTube =[test_YouTube, cont_YouTube]
Organic =[test_Organic, cont_Organic]

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([Email ,Adwords ,Facebook,YouTube ,Organic ], columns=['treatment', 'control'],index=["Email" ,"Adwords" ,"Facebook","YouTube" ,"Organic"])

## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([Email ,Adwords ,Facebook,YouTube ,Organic ])

## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ["Email" ,"Adwords" ,"Facebook","YouTube" ,"Organic"])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 4.5
P-value: 0.34
--------------------------
Expected Values
           treatment      control
Email     253.496072  8972.503928
Adwords   183.211772  6484.788228
Facebook   68.058722  2408.941278
YouTube    66.684908  2360.315092
Organic    33.548526  1187.451474
--------------------------
Observed Values
          treatment  control
Email           265     8961
Adwords         185     6483
Facebook         56     2421
YouTube          72     2355
Organic          27     1194
 


4-2. 2024-1-11전후로 가입한 고객중 구매자의 국가에 대한 차이가 없다
- 유의성검정 : P-value가 0.34로 0.05보다 크므로 귀무가설을 채택한다(차이가 없다)

5-1. 가입경로에 따른 가입자수 상관관계 

In [90]:
df['account_traffic_source'].unique()

array(['Search', 'Facebook', 'Organic', nan, 'Email', 'Display'],
      dtype=object)

In [91]:
# 실험군의 유입경로 가입자수
test_Email  = treatment[treatment['account_traffic_source']=='Email']['user_id'].nunique()
test_Display  = treatment[treatment['account_traffic_source']=='Display']['user_id'].nunique()
test_Facebook  =treatment[treatment['account_traffic_source']=='Facebook']['user_id'].nunique()
test_Search  =treatment[treatment['account_traffic_source']=='Search']['user_id'].nunique()
test_Organic  = treatment[treatment['account_traffic_source']=='Organic']['user_id'].nunique()

# 대조군의 유입경로 가입자수
cont_Email = control[control['account_traffic_source']=='Email']['user_id'].nunique()
cont_Display  = control[control['account_traffic_source']=='Display']['user_id'].nunique()
cont_Facebook = control[control['account_traffic_source']=='Facebook']['user_id'].nunique()
cont_Search = control[control['account_traffic_source']=='Search']['user_id'].nunique()
cont_Organic = control[control['account_traffic_source']=='Organic']['user_id'].nunique()


## Part1. Chi-square Test Report
Email = [test_Email, cont_Email]
Display =[test_Display, cont_Display]
Facebook =[test_Facebook, cont_Facebook]
Search =[test_YouTube, cont_YouTube]
Organic =[test_Organic, cont_Organic]

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([Email ,Display ,Facebook,Search ,Organic ], columns=['treatment', 'control'], index=["Email" ,"Display" ,"Facebook","Search" ,"Organic"])

In [92]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ["Email" ,"Display" ,"Facebook","Search" ,"Organic"])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 4.5
P-value: 0.34
--------------------------
Expected Values
           treatment      control
Email     253.496072  8972.503928
Display   183.211772  6484.788228
Facebook   68.058722  2408.941278
Search     66.684908  2360.315092
Organic    33.548526  1187.451474
--------------------------
Observed Values
          treatment  control
Email            85     3622
Display          84     3017
Facebook        131     4241
Search           72     2355
Organic         298    11123
 


- 유의성검정 : P-value가 0.34로 0.05보다 크므로 귀무가설을 채택한다(차이가 없다)

5-2. 2024-1-11전후로 가입한 고객중 구매자의 가입경로에 대해 차이가 없을 것이다

In [93]:
# 실험군의 유입경로 가입자수
test_Email  = treatment2[treatment2['account_traffic_source']=='Email']['user_id'].nunique()
test_Display  = treatment2[treatment2['account_traffic_source']=='Display']['user_id'].nunique()
test_Facebook  =treatment2[treatment2['account_traffic_source']=='Facebook']['user_id'].nunique()
test_Search  =treatment2[treatment2['account_traffic_source']=='Search']['user_id'].nunique()
test_Organic  = treatment2[treatment2['account_traffic_source']=='Organic']['user_id'].nunique()

# 대조군의 유입경로 가입자수
cont_Email = control2[control2['account_traffic_source']=='Email']['user_id'].nunique()
cont_Display  = control2[control2['account_traffic_source']=='Display']['user_id'].nunique()
cont_Facebook = control2[control2['account_traffic_source']=='Facebook']['user_id'].nunique()
cont_Search = control2[control2['account_traffic_source']=='Search']['user_id'].nunique()
cont_Organic = control2[control2['account_traffic_source']=='Organic']['user_id'].nunique()


## Part1. Chi-square Test Report
Email = [test_Email, cont_Email]
Display =[test_Display, cont_Display]
Facebook =[test_Facebook, cont_Facebook]
Search =[test_YouTube, cont_YouTube]
Organic =[test_Organic, cont_Organic]

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([Email ,Display ,Facebook,Search ,Organic ], columns=['treatment', 'control'], index=["Email" ,"Display" ,"Facebook","Search" ,"Organic"])

In [94]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ["Email" ,"Display" ,"Facebook","Search" ,"Organic"])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 4.5
P-value: 0.34
--------------------------
Expected Values
           treatment      control
Email     253.496072  8972.503928
Display   183.211772  6484.788228
Facebook   68.058722  2408.941278
Search     66.684908  2360.315092
Organic    33.548526  1187.451474
--------------------------
Observed Values
          treatment  control
Email            18      796
Display          20      651
Facebook         26      949
Search           72     2355
Organic          48     2357
 


5-2. 2024-1-11전후로 가입한 고객중 구매자의 가입경로에 대해 차이가 없다(결론)

6. 상위카테고리 구매에 대해 차이가 있을 것이다

In [95]:
# 2024-01-11 이후 주문량 TOP3
#df[df.order_created_at >= '2024-01-11'].category.value_counts()[:3].index
category_top3 = treatment.groupby('category')['order_items_id'].nunique().sort_values(ascending=False)[:3].index.to_list()
category_top3

['Intimates', 'Fashion Hoodies & Sweatshirts', 'Jeans']

In [96]:
print('[Chi-square Analysis Result Report]\n------------------------------------')
for i,category_name in enumerate(category_top3, start=1):

    test = treatment[treatment.category==category_name].user_id.nunique()
    ctrl = control[control.category==category_name].user_id.nunique()

    ## Part1. Chi-square Test Report
    purchase_category = [test, ctrl] # A와 B의 특정 category를 구매한 유저 수
    unpurchase_category = [treatment.user_id.nunique() - test, 
                          control.user_id.nunique() - ctrl]  # A와 B의 특정 category를 구매하지 않은 유저 수
    
    ## Part2. Contingency Table & Chi-squaure Model
    ## 기초 테이블 형성
    cont_table = pd.DataFrame([purchase_category, unpurchase_category], columns=['treatment', 'control'], index=[f'purchase_{category_name}', f'unpurchase_{category_name}'])
    
    ## Part3. 카이제곱 독립성 검정 모델 선언
    chi2, p_val, _, _= chi2_contingency([purchase_category, unpurchase_category])
    expected = chi2_contingency([purchase_category, unpurchase_category])[3]
    
    ## 기대값 표 형성
    ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = [f'purchase_{category_name}', f'unpurchase_{category_name}'])

    print(
      'Category {}: {}'.format(i, category_name)
    , 'Chi-square: {}'.format(round(chi2, 2))
    , 'P-value: {}'.format(round(p_val, 2))
    , '--------------------------'
    , 'Expected Values'
    , ex
    , '--------------------------'
    , 'Observed Values'
    , cont_table
    ,  '=========================='
    , sep = '\n'
    )

[Chi-square Analysis Result Report]
------------------------------------
Category 1: Intimates
Chi-square: 1.5
P-value: 0.22
--------------------------
Expected Values
                        treatment       control
purchase_Intimates     159.839752   5573.160248
unpurchase_Intimates  1948.160248  67926.839752
--------------------------
Observed Values
                      treatment  control
purchase_Intimates          175     5558
unpurchase_Intimates       1933    67942
Category 2: Fashion Hoodies & Sweatshirts
Chi-square: 2.54
P-value: 0.11
--------------------------
Expected Values
                                            treatment       control
purchase_Fashion Hoodies & Sweatshirts     146.178235   5096.821765
unpurchase_Fashion Hoodies & Sweatshirts  1961.821765  68403.178235
--------------------------
Observed Values
                                          treatment  control
purchase_Fashion Hoodies & Sweatshirts          165     5078
unpurchase_Fashion Hoodies & Sweatshi

6. 상위 카테고리에 따라 주문에 대한 차이가 없다

In [ ]:
7. 상위 주문브랜드에 대해 차이가 있을 것이다

In [100]:
# 전체 brand 주문량 TOP5
order_total_brand = df.groupby('brand').agg({'order_items_id':'nunique', 'category':'unique'}).reset_index().sort_values(by='order_items_id', ascending=False)[:5]
order_total_brand

,brand,order_items_id,category
96,Allegra K,2896,"[Pants & Capris, Dresses, Shorts, Pants, Sweat..."
447,Calvin Klein,1477,"[Dresses, Underwear, Blazers & Jackets, Jeans,..."
465,Carhartt,1166,"[Outerwear & Coats, Accessories, Jeans, Plus, ..."
2583,Volcom,854,"[Tops & Tees, Shorts, Swim, Fashion Hoodies & ..."
1721,Nautica,822,"[Sleep & Lounge, Swim, Sweaters, Pants, Shorts..."


In [101]:
# 2024-01-11 이전 주문량 TOP5
order_control_brand = control.groupby('brand').agg({'order_items_id':'nunique', 'category':'unique'}).reset_index().sort_values(by='order_items_id', ascending=False)[:5]
order_control_brand

,brand,order_items_id,category
94,Allegra K,2821,"[Pants & Capris, Dresses, Shorts, Pants, Sweat..."
439,Calvin Klein,1433,"[Dresses, Underwear, Blazers & Jackets, Jeans,..."
455,Carhartt,1125,"[Outerwear & Coats, Accessories, Jeans, Plus, ..."
2525,Volcom,830,"[Tops & Tees, Shorts, Swim, Fashion Hoodies & ..."
1059,Hanes,798,"[Socks, Intimates, Plus, Underwear, Socks & Ho..."


In [102]:
# 2024-01-11 이후 주문량 TOP5
order_test_brand = treatment.groupby('brand').agg({'order_items_id':'count', 'category':'unique'}).reset_index().sort_values(by='order_items_id', ascending=False)[:5]
order_test_brand

,brand,order_items_id,category
33,Allegra K,75,"[Dresses, Fashion Hoodies & Sweatshirts, Blaze..."
142,Calvin Klein,44,"[Intimates, Underwear, Pants, Jeans, Suits & S..."
151,Carhartt,41,"[Jeans, Pants, Outerwear & Coats, Suits & Spor..."
551,Nautica,34,"[Sleep & Lounge, Underwear, Socks, Shorts, Acc..."
619,Quiksilver,29,"[Tops & Tees, Swim, Sweaters, Shorts, Jeans, F..."


In [103]:
# Brand TOP3 (user_id)
# 2024-01-11 이후 주문량 TOP3
# df[df.order_created_at >= '2024-01-11'].brand.value_counts()[:3].index
Brand_top3 = treatment.groupby('brand')['order_items_id'].nunique().sort_values(ascending=False)[:3].index.to_list()
Brand_top3

['Allegra K', 'Calvin Klein', 'Carhartt']

In [104]:
print('[Chi-square Analysis Result Report]\n------------------------------------')
for i,brand_name in enumerate(Brand_top3, start=1):

    test = treatment[treatment.brand==brand_name].user_id.nunique()
    ctrl = control[control.brand==brand_name].user_id.nunique()

    ## Part1. Chi-square Test Report
    purchase_brand = [test, ctrl] # A와 B의 특정 brand를 구매한 유저 수
    unpurchase_brand = [treatment.user_id.nunique() - test, 
                        control.user_id.nunique() - ctrl]  # A와 B의 특정 brand를 구매하지 않은 유저 수
    
    ## Part2. Contingency Table & Chi-squaure Model
    ## 기초 테이블 형성
    cont_table = pd.DataFrame([purchase_brand, unpurchase_brand], columns=['treatment', 'control'], index=[f'purchase_{brand_name}', f'unpurchase_{brand_name}'])
    
    ## Part3. 카이제곱 독립성 검정 모델 선언
    chi2, p_val, _, _= chi2_contingency([purchase_brand, unpurchase_brand])
    expected = chi2_contingency([purchase_brand, unpurchase_brand])[3]
    
    ## 기대값 표 형성
    ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = [f'purchase_{brand_name}', f'unpurchase_{brand_name}'])

    print(
      'Brand {}: {}'.format(i, brand_name)
    , 'Chi-square: {}'.format(round(chi2, 2))
    , 'P-value: {}'.format(round(p_val, 2))
    , '--------------------------'
    , 'Expected Values'
    , ex
    , '--------------------------'
    , 'Observed Values'
    , cont_table
    ,  '=========================='
    , sep = '\n'
    )

[Chi-square Analysis Result Report]
------------------------------------
Brand 1: Allegra K
Chi-square: 0.2
P-value: 0.66
--------------------------
Expected Values
                        treatment       control
purchase_Allegra K      79.348323   2766.651677
unpurchase_Allegra K  2028.651677  70733.348323
--------------------------
Observed Values
                      treatment  control
purchase_Allegra K           75     2771
unpurchase_Allegra K       2033    70729
Brand 2: Calvin Klein
Chi-square: 0.08
P-value: 0.77
--------------------------
Expected Values
                           treatment       control
purchase_Calvin Klein      40.705745   1419.294255
unpurchase_Calvin Klein  2067.294255  72080.705745
--------------------------
Observed Values
                         treatment  control
purchase_Calvin Klein           43     1417
unpurchase_Calvin Klein       2065    72083
Brand 3: Carhartt
Chi-square: 1.77
P-value: 0.18
--------------------------
Expected Values
         

7. 상뮈 브랜드 주문에 대해 차이가 없다

8. 가입 후 첫 결재일에 대해 차이가 있을 것이다

In [107]:
first_order = df.groupby('user_id').order_created_at.min().to_frame().merge(df[['user_id', 'account_created_at']], on='user_id', suffixes=('', '_first'))

In [108]:
first_order = first_order.assign(order_diff=first_order['order_created_at'] - first_order['account_created_at'])
first_order = first_order.assign(order_diff=first_order.order_diff.dt.days)

In [109]:
first_order.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102365 entries, 0 to 102364
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   user_id             102365 non-null  float64       
 1   order_created_at    82409 non-null   datetime64[ns]
 2   account_created_at  102365 non-null  datetime64[ns]
 3   order_diff          82409 non-null   float64       
dtypes: datetime64[ns](2), float64(2)
memory usage: 3.1 MB


In [110]:
# 실험군: 2024 신규 유입 유저 / 대조군: 기존 유저
first_order.loc[first_order.account_created_at >= '2024-01-11', 'grp'] = 'treatment'
first_order.loc[first_order.account_created_at < '2024-01-11', 'grp'] = 'control'

In [113]:
test_first_order = first_order[first_order.grp=='treatment'].order_diff
first_order_max = test_first_order.max()
ctrl_first_order = first_order[(first_order.grp=='control') & (first_order.order_diff <= first_order_max)].order_diff

In [114]:
var_test = np.var(test_first_order)
var_ctrl = np.var(ctrl_first_order)

print('지불 금액의 분산: 실험군 {:.2f}일 vs 대조군 {:.2f}일'.format(var_test, var_ctrl))

if (var_test > var_ctrl * 2) or (var_ctrl > var_test * 2):
    print('분산이 2배 이상 차이나므로, Welch\'s t-test')
    equal_var=False
else:
    print('분산이 같으므로, Student\'s t-test')
    equal_var=True

지불 금액의 분산: 실험군 0.79일 vs 대조군 3.09일
분산이 2배 이상 차이나므로, Welch's t-test


In [116]:
import scipy.stats as stats
t_stat, p_val = stats.ttest_ind(test_first_order.dropna(), 
                                ctrl_first_order.dropna(),
                                equal_var=equal_var)

print('t-statistic:', t_stat, '   p-value:', p_val)

if p_val < 0.05:
    print('두 그룹이 차이가 있다.')
else:
    print('두 그룹이 차이가 없다.')

t-statistic: -48.14997125398849    p-value: 0.0
두 그룹이 차이가 있다.
